<h1>Import Nessesary Libraries</h1>

In [6]:
import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image

<h1>Create Dataloaders</h1>

In [7]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

data_transforms = {
    'train':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize
    ]),
    'validation':
    transforms.Compose([
        transforms.Resize((10,10)),
        transforms.ToTensor(),
        normalize
    ]),
}

image_datasets = {
    'train': 
    datasets.ImageFolder('data/train', data_transforms['train']),
    'validation': 
    datasets.ImageFolder('data/validation', data_transforms['validation'])
}

dataloaders = {
    'train':
    torch.utils.data.DataLoader(image_datasets['train'],
                                batch_size=32,
                                shuffle=True, num_workers=4),
    'validation':
    torch.utils.data.DataLoader(image_datasets['validation'],
                                batch_size=32,
                                shuffle=False, num_workers=4)
}

<h1>Model setup</h1>

In [8]:
device = torch.device("cuda:0")
model = models.resnet18(pretrained=True).to(device)
    
for param in model.parameters():
    param.requires_grad = False   
    
model.fc = nn.Sequential(
               nn.Linear(512, 2)).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(),lr=0.001)

<h1>Train model</h1>

In [9]:
from copy import deepcopy

def train_model(model, criterion, optimizer, num_epochs=10):
    best_loss = np.infty
    early_stop_count = 0
    best_model = deepcopy(model)
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        for phase in ['train', 'validation']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                _, preds = torch.max(outputs, 1)
                running_loss += loss.detach() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(image_datasets[phase])
            epoch_acc = running_corrects.float() / len(image_datasets[phase])

            print('{} loss: {:.4f}, acc: {:.4f}'.format(phase,
                                                        epoch_loss.item(),
                                                        epoch_acc.item()))
        if epoch_loss.item() < best_loss:
            print('new best model')
            best_loss = epoch_loss.item()
            best_model = deepcopy(model)
            early_stop_count = 0
        else:
            early_stop_count += 1
        if early_stop_count >= 100:
            print('no improvement, training stopped')
            break
    return best_model

model_trained = train_model(model, criterion, optimizer, num_epochs=10)

Epoch 1/10
----------
train loss: 0.5998, acc: 0.6567
validation loss: 0.6609, acc: 0.6154
new best model
Epoch 2/10
----------
train loss: 0.3200, acc: 0.9399
validation loss: 0.6048, acc: 0.5385
new best model
Epoch 3/10
----------
train loss: 0.1917, acc: 0.9657
validation loss: 0.5785, acc: 0.5385
new best model
Epoch 4/10
----------
train loss: 0.1202, acc: 0.9871
validation loss: 0.5503, acc: 0.5385
new best model
Epoch 5/10
----------
train loss: 0.0929, acc: 0.9957
validation loss: 0.5343, acc: 0.6154
new best model
Epoch 6/10
----------
train loss: 0.0782, acc: 0.9957
validation loss: 0.5193, acc: 0.6923
new best model
Epoch 7/10
----------
train loss: 0.0493, acc: 1.0000
validation loss: 0.5041, acc: 0.6923
new best model
Epoch 8/10
----------
train loss: 0.0473, acc: 1.0000
validation loss: 0.4924, acc: 0.6923
new best model
Epoch 9/10
----------
train loss: 0.0455, acc: 0.9957
validation loss: 0.4885, acc: 0.7692
new best model
Epoch 10/10
----------
train loss: 0.0389, acc

<h1>Save model</h1>

In [10]:
torch.save(model_trained.state_dict(), 'weights.h5')